# Parameter definition
Choose all the settings for the database creation.

In [5]:
#TODO: Change absolute to relative paths for input and output dir.
import os

home = os.path.expanduser("~")

db_name= 'test_video_dataset' #'colab_main_dataset'
input_dir = os.path.join(home,'image_datasets')
output_dir = os.path.join(home,'sql_database')
database_names=['lfw'] # , 'scface', 'forenface', 'enfsi', 'enfsi2015'
detector_names=['mtcnn', 'mtcnn_serfiq'] # 'dlib', 'mtcnn', 'mtcnn_serfiq'
embedding_model_names=["ArcFace"] # , "Dlib", "ArcFace",
attributes_to_update=['gender', 'age']  # 'gender', 'age', 'emotion', 'race'
quality_model_names = ['ser_fiq'] # 'confusion_score',
save_in_drive = False # To work in google colab



## SERFIQ model

Create SERFIQ model.

In [6]:
from sql_face.serfiq import get_serfiq_model
serfiq = get_serfiq_model()

[14:10:51] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v1.8.0. Attempting to upgrade...
[14:10:51] ../src/nnvm/legacy_json_util.cc:216: Symbol successfully upgraded!
[14:10:52] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v0.8.0. Attempting to upgrade...
[14:10:52] ../src/nnvm/legacy_json_util.cc:216: Symbol successfully upgraded!
[14:10:52] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v0.8.0. Attempting to upgrade...
[14:10:52] ../src/nnvm/legacy_json_util.cc:216: Symbol successfully upgraded!
[14:10:52] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v0.8.0. Attempting to upgrade...
[14:10:52] ../src/nnvm/legacy_json_util.cc:216: Symbol successfully upgraded!
[14:10:52] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v0.8.0. Attempting to upgrade...
[14:10:52] ../src/nnvm/legacy_json_util.cc:216: Symbol successfully upgraded!


# Database creation

database = SQLDataBase(db_name=db_name,
                        database_names=database_names, # , 'scface', 'forenface', 'enfsi', 'enfsi2015'
                        detector_names=detector_names,
                        embedding_model_names=embedding_model_names,
                        quality_model_names=quality_model_names,
                        save_in_drive=save_in_drive                        
                        )

database.fill_db()
database.update_db(attributes_to_update=attributes_to_update, force_update=False)

In [7]:
from sql_face.sqldb import SQLDataBase

In [8]:
database = SQLDataBase(db_name=db_name,
                        input_dir = input_dir,
                        output_dir_name = output_dir,
                        database_names = database_names,
                        detector_names = detector_names,
                        embedding_model_names = embedding_model_names,
                        quality_model_names = quality_model_names)

Creating Db file at /home/jmacarulla/sql_database/test_video_dataset.db


In [9]:
database.__dict__

{'db_name': 'test_video_dataset',
 'input_dir': '/home/jmacarulla/image_datasets',
 'save_in_drive': False,
 'output_dir': '/home/jmacarulla/sql_database',
 'session': <sqlalchemy.orm.session.Session at 0x7fb8e6d94e10>,
 'databases': [<sql_face.databases.LFW at 0x7fb8e6d4e410>],
 'detector_names': ['mtcnn', 'mtcnn_serfiq'],
 'embedding_model_names': ['ArcFace'],
 'quality_model_names': ['ser_fiq']}

In [10]:
database.create_tables(serfiq=serfiq)

Creating image record from LFW: 13233it [00:00, 14786.93it/s]
TRIM Creating Cropped Images for detector mtcnn:   0%|          | 0/5 [00:00<?, ?it/s]2022-11-13 14:14:44.968694: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


1/1 [==============================] - 0s 165ms/step


TRIM Creating Cropped Images for detector mtcnn:  20%|██        | 1/5 [02:16<09:05, 136.32s/it]

1/1 [==============================] - 0s 29ms/step


TRIM Creating Cropped Images for detector mtcnn:  40%|████      | 2/5 [02:24<03:02, 60.90s/it] 

...

TRIM Creating Cropped Images for detector mtcnn:  60%|██████    | 3/5 [02:31<01:12, 36.46s/it]

1/1 [==============================] - 0s 27ms/step


TRIM Creating Cropped Images for detector mtcnn:  80%|████████  | 4/5 [02:40<00:25, 25.32s/it]

1/1 [==============================] - 0s 26ms/step


TRIM Creating Cropped Images for detector mtcnn: 100%|██████████| 5/5 [02:46<00:00, 33.28s/it]
TRIM Creating Cropped Images for detector mtcnn_serfiq:   0%|          | 0/5 [00:00<?, ?it/s][14:17:36] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.
[14:17:37] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.
[14:17:37] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.
[14:17:37] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.
[14:17:37] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.
[14:17:37] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.
[14:17:37] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.
[14:17:37] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.
[14:17:37] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.
[14:17:37] 

In [ ]:
database.update_tables(attributes_to_update, serfiq = serfiq)

Computing SER-FIQ quality:   0%|          | 0/197 [00:00<?, ?it/s]

1/1 [==============================] - 0s 60ms/step


Computing SER-FIQ quality:   0%|          | 0/197 [00:13<?, ?it/s]


KeyboardInterrupt: 